In [1]:
# Order preparation for SnP

# STATUS: Incomplete
# Run-time: 10 seconds

# Dependencies:
# /zdata/pkls/*.pkl - for pickles generated by 01_snp_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7497, clientId=11) # rkv tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=11) # rkv IBG live

In [6]:
%%time
import pandas as pd
import numpy as np
import datetime
from os import listdir

#... set limits and penalties

m_maxp = 0.015     # % of max margin allowed on net liquidity per scrip to limit position risk

min_rom = 0.45      # 0.5 would be 50% return

base = 0.01        # Upper or lower base multiple for prices

max_nlvp = 0.7     # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                   # max available funds for option trades = max_nlvp * NLV - initMargin
    
max_dte = 65       # no more than 65 days

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#****    PREPARE TO HARVEST   ****
#_________________________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

#...identify Stock and Option
rights = [s.right for s in df_p.contract]
df_p['Type'] = ['Stock' if r == '0' else 'Option' for r in rights]

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

#***   Error catching for list comprehension ***
#_______________________________________________
def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher
    Args: 
        (func) as the function
         (handle) as the lambda of function
         <*args | *kwargs> as arguments to the functions
    Outputs:
        output of the function | <np.nan> on error
    Usage:
        eggs = [1,3,0,3,2]
        [catch(lambda: 1/egg) for egg in eggs]'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
#     if dte is to be extracted from contract.lastTradeDateOrContractMonth
    try:
        dte = (util.parseIBDatetime(expiry) - datetime.datetime.now().date()).days
    except Exception as e:
        return np.nan
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]

discount = [m for m in map(expPricePct, expiry)]

df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 1 cent

df_hvst = df_p[df_p.hvstPrice.notnull()]

# harvest open positions with hvstPrice
qual_contracts = ib.qualifyContracts(*df_hvst.contract)

# df_hvst = df_hvst.assign(qual_contracts = qual_contracts)

df_hvst = df_hvst.assign(harvestOrder = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_hvst.position, df_hvst.hvstPrice)])

Wall time: 28.9 ms


In [ ]:
#****   PREPARE TO SOW !    ****
#________________________________

#...make the blacklist
df_p['obj_type'] = [type(s).__name__ for s in df_p.contract]

 # margin of contract
order = [Order(action='SELL', totalQuantity=1, orderType='MKT') for _ in df_p.items()]

df_p['margin'] = [float(ib.whatIfOrder(*i).initMarginChange) for i in zip(df_p.contract, order)]*df_p.position

df1 = df_p.groupby('ibSymbol').sum()

df1['avail_margin'] = df1.margin + max_p

df1['max_units'] = [int(i) for i in df1.avail_margin/(df1.marketPrice*100)]

blacklist = list(df1[df1.max_units <= 0].index)

#... build the high-pop-roc dataframe

# read the dataframe pickles from zdata. Ignore the underscores (underlying)
fs = listdir('./zdata/')
fs = [f for f in fs if (f[-3:] == 'pkl') & (f[0] != '_')]
dfs = pd.concat([pd.read_pickle(r'./zdata/'+f) for f in fs], axis=0).reset_index(drop=True).sort_values('rom', ascending=False)

# filter out puts which are not in blacklist
dfs_p = dfs[(dfs.right=='P') & (~dfs.symbol.isin(blacklist))]

# filter out puts whose underlying price is greater than minimum rom
df2 = dfs_p[(dfs_p.rom > min_rom)]

# filter out max_dte
df3 = df2[df2.dte <= max_dte]

# filter puts whose underlying price is above the mean.
# df3 = df2[(df2.undPrice > df2.avg) & (df2.strike < (df2.avg - df2.stDev*sigma))]

In [ ]:
df3[['symbol', 'strike', 'dte', 'right', 'undPrice', 'pop', 'rom', 'expPrice', 'hv', 'bsmDelta', 'bsmPrice', 'ibiv', 'ibdelta', 'ibprice', 'close', 'bid', 'ask', 'lo', 'avg']]

In [ ]:
# WARNING!!!... Make df the dataframe that you want to execute on!
df = df3.copy()   # make this the last dataframe to get the orders placed

contracts = [c.contract for c in df.opt_ticker]

df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df.lot, df.expPrice)]

print('{:d} contracts from {:d} scrips, consuming {:,} margin from full available funds of {:,}'.format(len(contracts), len(df.symbol.unique()), sum(df.margin), av_funds))

In [ ]:
df_analyze = df[df.symbol.isin(df.symbol.unique())].groupby(['dte', 'symbol', 'strike']).apply(max)

df_analyze.to_excel('./zdata/analyze.xlsx', index=None, header=True)

# Write to watchlist. This watchlist is to be checked in tws / tradingview for the lowest strike and expiry
# This needs to be imported to IBKR's watchlist
watchlist = [('DES', s, 'STK', 'SMART') for s in df.symbol.unique()]
df_watch = util.df(watchlist)

df_watch.to_csv('./zdata/watchlist.csv', index=None, header=False)